# CaloCem 

CaloCem is a package for processing cement calorimetry data. https://github.com/mj-hofmann/CaloCem. Currently, it can only process TA Instrument type calorimetry files.

Here, we are demonstrating basic functionality.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tgaedt/workshop_uv/blob/main/notebooks/3_calocem.ipynb)



In [ ]:
!pip install calocem

In [ ]:
%%bash

git clone --depth 1 --filter=blob:none --sparse https://github.com/tgaedt/workshop_uv.git

cd workshop_uv
git sparse-checkout set data

In [ ]:
import calocem.tacalorimetry as cct
from pathlib import Path
import matplotlib.pyplot as plt

calodatapath = Path("workshop_uv/data/calo/")
metadata = Path("workshop_uv/data/metadata_calo/metadata.csv")


tam = cct.Measurement(calodatapath, processed=True, new_code=True)

In [ ]:
tam.plot()

In [ ]:
tam.add_metadata_source(metadata, sample_id_column="sample")

plots = tam.plot_by_category(categories="additive_name")

for plot in plots:
    plt.show()

In [ ]:
from calocem.processparams import ProcessingParameters
processparams = ProcessingParameters()

processparams.spline_interpolation.apply = True
processparams.spline_interpolation.smoothing_1st_deriv = 1e-11
# processparams.gradient_peakdetection.use_largest_width_heigth = True
processparams.gradient_peakdetection.prominence = 1e-7

tam.get_maximum_slope(processparams=processparams, show_plot=True)


In [ ]:
onsets = tam.get_peak_onset_via_max_slope(processparams=processparams, show_plot=True)

In [ ]:
import pandas as pd

onsets = pd.merge(onsets, tam._metadata, left_on="sample_short", right_on="sample")
onsets

In [ ]:
for samplename, sampledata in onsets.groupby("additive_name"):
    fig, ax = plt.subplots()
    ax.scatter(sampledata["dosage"], sampledata["onset_time_s"])
    ax.set_title(samplename)
    plt.show()

In [ ]:
def create_reference_data(dfm, sample_code, retarders):
    data = dfm.copy()
    mask = data["additive_name"] == sample_code
    data.loc[mask, "additive_name"] = data.loc[mask, "additive_name"].apply(lambda x: retarders)
    data = data.explode("additive_name")
    return data

onsets_ref = create_reference_data(onsets, "reference", ["sucrose", "glucose"])



In [ ]:
fig, ax = plt.subplots()

for samplename, sampledata in onsets_ref.groupby("additive_name"):
    ax.scatter(sampledata["dosage"], sampledata["onset_time_s"], label=samplename)
    ax.set_xlabel("Dosage [µmol/g]")
    ax.set_ylabel("Onset Time [s]")
    ax.legend()